In [1]:
import os
import plotly.express as px
import pandas as pd
import numpy as np
import json

from utils import *
from train import *
from evaluation import *
import wandb
wandb.login()

import warnings
warnings.filterwarnings('ignore')

# Set seed
np.random.seed(42)

# Set working directory
os.chdir(r"..") # should be the git repo root directory, checking below:
print("Current working directory: " + os.getcwd())
assert os.getcwd()[-8:] == "WattCast"
dir_path = os.path.join(os.getcwd(), 'data', 'clean_data')
model_dir = os.path.join(os.getcwd(), 'models')



/Users/nikolaushouben/opt/anaconda3/envs/Wattcast/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nikolaushouben (wattcast). Use `wandb login --relogin` to force relogin


Current working directory: /Users/nikolaushouben/Desktop/WattCast


In [2]:
def train_eval_tuning():

    wandb.init(project="WattCast_tuning")
    wandb.config.update(config_run)
    config = wandb.config

    print("Getting data...")

    pipeline, ts_train_piped, ts_val_piped, ts_test_piped, ts_train_weather_piped, ts_val_weather_piped, ts_test_weather_piped, trg_train_inversed, trg_val_inversed, trg_test_inversed = data_pipeline(config)

    print("Getting model instance...")
    model_instance = get_model(config)
    model_instance, _ = train_models([model_instance], ts_train_piped, ts_train_weather_piped, ts_val_piped, ts_val_weather_piped)

    print("Evaluating model...")
    predictions, score = predict_testset(model_instance[0], 
                                  ts_test_piped[config.longest_ts_test_idx], 
                                  ts_test_weather_piped[config.longest_ts_test_idx],
                                  config.n_lags, config.n_ahead, config.eval_stride, pipeline,
                                  )


    print("Plotting predictions...")
    df_compare = pd.concat([trg_test_inversed.pd_dataframe(), predictions], axis=1).dropna()
    df_compare.columns = ['target', 'prediction']
    fig = px.line(df_compare, title='Predictions vs. Test Set')

    wandb.log({'eval_loss': score})
    wandb.log({'predictions': fig})
    wandb.finish()



### Running

In [ ]:
#we keep the number of sweeps constant to ensure a fair playing field for all algorithms
sweeps = 3 # number of sweeps per model, per location: 

scale_location_pairs = (
     ('1_county', 'Sacramento'),
    ('1_county', 'New_York'),
    #('1_county', 'Los_Angeles'),
    ('2_town', 'town_0'),
     ('2_town', 'town_1'),
     ('2_town', 'town_2'),
     ('3_village', 'village_1'),
     ('3_village', 'village_2'),
    ('3_village', 'village_0'),
    ('4_neighborhood', 'neighborhood_0'),
     ('4_neighborhood', 'neighborhood_1'),
     ('4_neighborhood', 'neighborhood_2'),
    ('5_building', 'building_0'),
    ('5_building', 'building_1'),
    ('5_building', 'building_2'),
      )

scale_location_pairs = (
     ('1_county', 'Sacramento'),
)

models = [
        #'rf',
        #'xgb', 
        #'gru', 
        #'lgbm',  
        'nbeats',
        #'tft'
        ]


for scale, location in scale_location_pairs:

    for model in models:
        # placeholder initialization of config file (will be updated in train_eval_light())
        config_run = {
            'spatial_scale': scale,
            'temp_resolution': 60,
            'location': location,
            'model': model,
            'horizon_in_hours': 24,
            'lookback_in_hours': 24,
            'boxcox': True,
            'liklihood': None,
            'weather': True,
            'holiday': True,
            'datetime_encodings': False,
            'heat_wave_binary': False,
        }
        
        with open(f'sweep_configurations/config_sweep_{model}.json', 'r') as fp:
            sweep_config = json.load(fp)                  

        sweep_config['name'] = model + 'sweep' + config_run['spatial_scale'] + '_' + config_run['location'] + '_' + str(config_run['temp_resolution'])

        sweep_id = wandb.sweep(sweep_config, project="WattCast_tuning")
        wandb.agent(sweep_id, train_eval_tuning, count=sweeps)
        wandb.finish()



Create sweep with ID: yoi3jsof
Sweep URL: https://wandb.ai/wattcast/Wattcast_tuning/sweeps/yoi3jsof


wandb: Agent Starting Run: uuku2864 with config:
wandb: 	datetime_encodings: 1
wandb: 	dropout: 0
wandb: 	heat_wave_binary: 0
wandb: 	layer_widths: 1024
wandb: 	lr: 0.01
wandb: 	num_blocks: 1
wandb: 	num_layers: 8
wandb: 	num_stacks: 50
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'datetime_encodings' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'heat_wave_binary' was locked by 'sweep' (ignored update).


Getting data...


Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.


...

wandb: WARNING Config item 'datetime_encodings' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'heat_wave_binary' was locked by 'sweep' (ignored update).


Getting data...


Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.


Getting model instance...
Training NBEATSModel


Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.


### Degbugging

In [ ]:
from train import Config

In [ ]:
config_run = {
    'spatial_scale': '1_county',
    'temp_resolution': 60,
    'location': 'Los_Angeles',
    'model': 'tft',
    'horizon_in_hours': 24,
    'lookback_in_hours': 24,
    'boxcox': True,
    'liklihood': None,
    'weather': True,
    'holiday': True,
    'datetime_encodings': False,
    'heat_wave_binary': False,
}

wandb.init(project="WattCast_tuning")
wandb.config.update(config_run)
config = wandb.config



In [ ]:
config

In [ ]:


config = Config()

config = config.from_dict(config_run)


with open(f'sweep_configurations/config_sweep_{config.model}.json', 'r') as fp:
            sweep_config = json.load(fp)   

hypers = {param: values['values'][0] for param, values in sweep_config['parameters'].items()}

for param, value in hypers.items():
    setattr(config, param, value)


In [ ]:
pipeline, ts_train_piped, ts_val_piped, ts_test_piped, ts_train_weather_piped, ts_val_weather_piped, ts_test_weather_piped, trg_train_inversed, trg_val_inversed, trg_test_inversed = data_pipeline(config)


In [ ]:
m = get_model(config)